### In this notebook I want to rerun the the A2 colour plot whilst keeping the degeneracy across equal parities but finding the minimum energy gap between the low energy parity and the first excited state which isn't degenerate

We are using the same structure as reported in Nick's Circuit Design paper. 4 ancilla bits and 4 logical bits used to construct an eefecting 4 body coupling. We will apply 2 different schedules to the transverse field so that we have $A_{ancilla}(s)$ and $A_{logical}(s)$ but they will share the same $B(s)$.

In [ ]:
#degeneracy of lowest level should be
from scipy.misc import factorial
def deg(N):
    n_d = np.arange(1, N+N%2,2)
    n_u = -n_d+N
    return factorial(N, exact = True)*sum(1./(factorial(n_d, exact = True)*factorial(n_u, exact = True) )  )

print deg(4)

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\circuits_paper_connection_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

In [ ]:
# have a signle run before we move on to do the full scope examination

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 1.
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it
a = mylib.Anneal(qubits,[h,J],T=100,points = 1000)
a.run()
a.show_results()

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Jup010 - Anneal_path_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

I want to carry out a bunch of anneals where the annealing schedule A path with have the above form. I want them to be simply a linear schedule where they are related by
\begin{equation}
    A_{anc}(s)=(1-s) \sin(\theta)
\end{equation}

\begin{equation}
    A_{log}(s)=(1-s) \cos(\theta)
\end{equation}

\begin{equation}
    \text{and so }\\
    \sqrt{A_{anc}^2(s)+A_{log}^2(s)}=(1-s)
\end{equation}

In [ ]:
# I should have used this instead. The index 100 isn't a good choice since cos(theta)=0 for this
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))
print thetas_unordered

In [ ]:
# run with the non-neutral hamiltonian and removing degeneracy

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 1.
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200
an = {}
last_prob = []
for x in thetas_unordered:
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter021.001')

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
#      HERE I WANT TO DEFINE A FUNCTION THAT RETURNS THE HAMILTONIAN AT ANY POINT ON A2 SPACE
# ------------------------------------------------------------------------------------------------------------------------------
'''We need this to be specific to the linear schedule. So for each point we will need to calculate the angle of theta and
thus calculate s so we can feed this into B(s). 
I want to be able create a plot that has sqrt(A_log**2+A_anc**2) = 1 since these are the point that my previous work has 
passed through'''
nax = 1000
sp = np.linspace(0, 1, nax)
gaps = np.zeros((sp.size,sp.size))

def H_A2((log,anc),s):
    return sum([a.Hb[i]*A for i,A in enumerate([log]*log_qub+[anc]*anc_qub)])+s*a.Hp

# for the following function i assume that the eigenvalues are always given in increasing order
def gap_finder(M,Deg):
    '''M is the hamiltonian matrix and D is the degeneracy'''
    d,P,P_in = mylib.Diagonaliser(M)
    return d[Deg]-d[Deg-1]

def gap_A2((log,anc),Deg):
    s = 1-(anc**2+log**2)
    if s<0: return 0
    else: return gap_finder(H_A2((log,anc),s),Deg)

Deg = int(deg(log_qub))
for x,log in enumerate(sp):
    print 'running %s of %s'%(x,nax-1)
    for y,anc in enumerate(sp):
        gaps[x,y] = gap_A2((log,anc),Deg)
        
mylib.save_object(gaps,'Jupyter021.002')

In [ ]:
# want to make the data of the same format that I did for the other A2 colour plot
thetas = np.asarray([float(key[6:]) for key in an.keys()])
reorder = thetas.argsort()
thetas = thetas[reorder]
last_probs = np.asarray([an['theta='+str(theta)].problem_x0_prob[-1] for theta in thetas])

obj = {'thetas': thetas, 'last_probs': last_probs}
mylib.save_object(obj,'Jupyter020.003')


In [ ]:
plt.figure()
plt.plot(thetas*(90*2/np.pi), last_probs)
plt.ylabel(r'$P$(success)')
plt.xlabel(r'$\theta$ in degrees')
plt.show()

In [ ]:
max_success_theta = thetas[last_probs == max(last_probs)][0]
an['theta=%s'%max_success_theta].show_results()